# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [4]:
df.shape

(77536, 9)

In [5]:
df.columns

Index(['loan_size', 'interest_rate', 'homeowner', 'borrower_income',
       'debt_to_income', 'num_of_accounts', 'derogatory_marks', 'total_debt',
       'loan_status'],
      dtype='object')

# Data-Pre-Processing

### Part 1: Define X & y components. Label Encoding.

In [6]:
# Label Encoder for non-numeric feature columns. 
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()   

# Fit the label encoder with non-numeric column.
label_encoder.fit(df['homeowner'])                             # homeowner is the only column that's non-numeric. 

# Add a new features column to X, transforming the homeowner values to number classes. 
df['homeowner_le'] = label_encoder.transform(df['homeowner'])

#Drop the original homeowner column
df.drop(['homeowner'], axis=1, inplace=True)

In [7]:
# Check the label_encoder classes
list(label_encoder.classes_)

['mortgage', 'own', 'rent']

In [8]:
df.columns

Index(['loan_size', 'interest_rate', 'borrower_income', 'debt_to_income',
       'num_of_accounts', 'derogatory_marks', 'total_debt', 'loan_status',
       'homeowner_le'],
      dtype='object')

### Part 2: Split the Data into Training and Testing

In [9]:
# Create our features
X = df.copy() 
X.drop('loan_status', axis=1, inplace=True)

# Create our target
y = df['loan_status']       # The loan_status column is our dependent vector to predict. 

In [10]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_le
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.606144
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.667811
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,1.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,2.000000


In [11]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [14]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Part 3: Standard Scaler and Scaling the Data

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [15]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()   # Create scaler object. 

In [16]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = data_scaler.fit(X_train)       # You have to do LabelEncoding of non-numeric values or this won't work. 

In [17]:
# Scale the FEATURES training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [18]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(solver='lbfgs', random_state=1)
lr_model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [19]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred_lr = lr_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred_lr)

0.9520479254722232

In [44]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_lr)
cm_df = pd.DataFrame(
    cm,
    index = ['Actual 0', 'Actual 1'],
    columns = ['Predicted 0', 'Predicted 1']
)

cm_df

,Predicted 0,Predicted 1
Actual 0,563,56
Actual 1,102,18663


In [21]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_lr))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      0.91      0.99      0.88      0.95      0.90       619
   low_risk       1.00      0.99      0.91      1.00      0.95      0.91     18765

avg / total       0.99      0.99      0.91      0.99      0.95      0.91     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [22]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state = 1)
X_resampled_ros, y_resampled_ros = ros.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled_ros)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [23]:
# Train the Logistic Regression model using the resampled data
ros_model = LogisticRegression(solver='lbfgs', max_iter =2000, random_state=1)
ros_model.fit(X_resampled_ros, y_resampled_ros)

LogisticRegression(max_iter=2000, random_state=1)

In [24]:
# Calculated the balanced accuracy score
y_pred_ros = ros_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred_ros)

0.9936781215845847

In [46]:
# Display the confusion matrix
cm_ros = confusion_matrix(y_test, y_pred_ros)
cm_ros_df = pd.DataFrame(
    cm_ros,
    index = ['Actual 0', 'Actual 1'],
    columns = ['Predicted 0', 'Predicted 1']
)

cm_ros_df

,Predicted 0,Predicted 1
Actual 0,615,4
Actual 1,116,18649


In [26]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred_ros))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [27]:
# Resample the training data with SMOTE
# YOUR CODE HERE
from imblearn.over_sampling import SMOTE
X_resampled_sm, y_resampled_sm = SMOTE(random_state = 1, sampling_strategy = 1.0).fit_resample(X_train, y_train)

# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled_sm)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [28]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
sm_model = LogisticRegression(solver='lbfgs', max_iter =2000, random_state=1)
sm_model.fit(X_resampled_sm, y_resampled_sm)

LogisticRegression(max_iter=2000, random_state=1)

In [29]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred_sm = sm_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred_sm)

0.9936781215845847

In [47]:
# Display the confusion matrix
# YOUR CODE HERE
cm_sm = confusion_matrix(y_test, y_pred_sm)
cm_sm_df = pd.DataFrame(
    cm_sm,
    index = ['Actual 0', 'Actual 1'],
    columns = ['Predicted 0', 'Predicted 1']
)

cm_sm_df

,Predicted 0,Predicted 1
Actual 0,615,4
Actual 1,116,18649


In [31]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred_sm))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [34]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state = 1)
X_resampled_cc, y_resampled_cc = cc.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled_cc)

Counter({'high_risk': 1881, 'low_risk': 1881})

In [35]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
cc_model = LogisticRegression(solver = 'lbfgs', random_state = 1)
cc_model.fit(X_resampled_cc, y_resampled_cc) 

LogisticRegression(random_state=1)

In [36]:
# Calculate the balanced accuracy score
y_pred_cc = cc_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred_cc)

0.9865149130022852

In [48]:
# Display the confusion matrix
cm_cc = confusion_matrix(y_test, y_pred_cc)
cm_cc_df = pd.DataFrame(
    cm_cc,
    index = ['Actual 0', 'Actual 1'],
    columns = ['Predicted 0', 'Predicted 1']
)

cm_cc_df

,Predicted 0,Predicted 1
Actual 0,606,13
Actual 1,112,18653


In [38]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_cc))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.98      0.99      0.91      0.99      0.97       619
   low_risk       1.00      0.99      0.98      1.00      0.99      0.97     18765

avg / total       0.99      0.99      0.98      0.99      0.99      0.97     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [39]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
cos = SMOTEENN(random_state = 1)
X_resampled_cos, y_resampled_cos = cos.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled_cos)

Counter({'high_risk': 55603, 'low_risk': 55948})

In [40]:
# Train the Logistic Regression model using the resampled data
cos_model = LogisticRegression(solver = 'lbfgs', random_state = 1)
cos_model.fit(X_resampled_cos, y_resampled_cos) 

LogisticRegression(random_state=1)

In [41]:
# Calculate the balanced accuracy score
y_pred_cos = cos_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred_cos)

0.9935182494822666

In [49]:
# Display the confusion matrix
cm_cos = confusion_matrix(y_test, y_pred_cos)
cm_cos_df = pd.DataFrame(
    cm_cos,
    index = ['Actual 0', 'Actual 1'],
    columns = ['Predicted 0', 'Predicted 1']
)

cm_cos_df

,Predicted 0,Predicted 1
Actual 0,615,4
Actual 1,122,18643


In [43]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_cos))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   All machine learning model methods, after resampling the training data, produced the same balanced accuracy score of **99.3**

2. Which model had the best recall score?

    All machine learning learning models had the same recall rate of **99%**

3. Which model had the best geometric mean score?

    
